In [2]:
import pandas as pd
import collections

In [4]:
df=pd.read_excel('data/small_used_car.xlsx')
df.head()

,mileage,model,price,year,my_car_damage,other_car_damage
0,63608,K3,970,2017,0,564596
1,69336,K3,1130,2015,1839700,1140150
2,36000,K3,1380,2016,446520,2244910
3,19029,K3,1390,2017,889000,4196110
4,97090,K3,760,2015,2339137,2029570


### 더미코딩

In [11]:
pd.get_dummies(df)

,mileage,price,year,my_car_damage,other_car_damage,model_Avante,model_K3
0,63608,970,2017,0,564596,0,1
1,69336,1130,2015,1839700,1140150,0,1
2,36000,1380,2016,446520,2244910,0,1
3,19029,1390,2017,889000,4196110,0,1
4,97090,760,2015,2339137,2029570,0,1
...,...,...,...,...,...,...,...
269,235000,390,2007,1473730,507260,1,0
270,140000,430,2009,14399366,4592589,1,0
271,96757,390,2006,303080,0,1,0
272,113853,390,2008,320780,1857718,1,0


In [16]:
df['md'] = df['model'].map({'K3': 1, 'Avante': 0})

## 예측하기

In [15]:
import numpy
from statsmodels.formula.api import logit

In [18]:
res = logit('md ~ mileage + price + year',df).fit()

Optimization terminated successfully.
         Current function value: 0.539595
         Iterations 7


In [19]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     md   No. Observations:                  274
Model:                          Logit   Df Residuals:                      270
Method:                           MLE   Df Model:                            3
Date:                Fri, 23 Oct 2020   Pseudo R-squ.:                 0.04383
Time:                        11:42:07   Log-Likelihood:                -147.85
converged:                       True   LL-Null:                       -154.63
Covariance Type:            nonrobust   LLR p-value:                  0.003578
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -643.2888    215.572     -2.984      0.003   -1065.801    -220.776
mileage    -4.779e-06   4.93e-06     -0.969      0.332   -1.44e-05    4.89e-06
price         -0.0016      0.001     -1.827      0.068      -0.003       0.000
year           0.3198      0.107      2.980      0.003       0.109       0.530
==============================================================================
"""

In [27]:
predict=res.predict(df)
prediction = numpy.where(predict > 0.1,1,0)

## 혼동 행렬 (confusion matrix)

In [28]:
from sklearn.metrics import confusion_matrix

In [31]:
confusion_matrix(df['md'],prediction)

array([[ 12, 193],
       [  0,  69]])

## 정확도 (Accuracy)

(TP + TN / Total)

In [32]:
tp = 180
tn = 240
fp = 60
fn = 20

In [33]:
(tp+tn)/(tp+tn+fp+fn)

0.84

## 정밀도 (precision)

tp / (tp + fp)

In [34]:
tp / (tp+fp)

0.75

## 재현도 (Recall)

In [35]:
tp / (tp+fn)

0.9